In [4]:
import xarray as xr
import pandas as pd
import numpy as np
from bbox import * 

lat_min = 28.6
lat_max = 33.4
lon_min = -98.9
lon_max = -88.3


In [5]:
new_lat_max = 33.39
new_lon_max = -88.31

In [8]:
o3_hours = pd.read_csv("../../data/tempo_data/valid_o3_hours.csv")
o3_hours=list(o3_hours['time'])
time_list = pd.to_datetime(o3_hours)
unique_months = time_list.to_period('M').unique()

In [12]:
unique_months

PeriodIndex(['2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01',
             '2024-02', '2024-03', '2024-04', '2024-05', '2024-06', '2024-07',
             '2024-08', '2024-09', '2024-10', '2024-11', '2024-12', '2025-01',
             '2025-02', '2025-03', '2025-04', '2025-05'],
            dtype='period[M]')

In [11]:
unique_months[-1:]

PeriodIndex(['2025-05'], dtype='period[M]')

In [7]:
## CREATE y_sensor_o3 before running!

In [13]:
for month in unique_months[-1:]:
    month_str = str(month)
    print(month_str)
    tempo = xr.open_dataset(f'../../data/tempo_data/o3_monthly_files/tempo_o3_{month_str}.nc')
    weather = xr.open_dataset(f'../../data/weather_data_samehr/weather_data_{month_str}.nc')
    # tempo = tempo.rename({'latitude': 'lat', 'longitude': 'lon'})
    # ✅ Rename latitude/longitude to lat/lon only if needed
    rename_map = {}
    if "latitude" in tempo.coords:
        tempo = tempo.rename({'latitude':'lat'})
    if "longitude" in tempo.coords:
        tempo = tempo.rename({'longitude':'lon'})

    tempo = tempo.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"{var}_tempo" for var in tempo.data_vars}
    tempo = tempo.rename(new_names)
    month_ts = np.array(tempo['time'])
    month_ts = np.array(list(set(list(month_ts))&set(list(np.array(weather['time'])))))
    tempo = tempo.sel(time=month_ts)
    tempo['lat'] = np.round(tempo['lat'].values, 3)
    tempo['lon'] = np.round(tempo['lon'].values, 3)

    # weather = xr.open_dataset(f'../../data/weather_data_samehr/weather_data_{month_str}.nc')
    weather = weather.drop_vars(['UGRD', 'VGRD'])
    weather = weather.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"{var}_weather" for var in weather.data_vars}
    weather = weather.rename(new_names)
    month_ts = np.array(list(set(list(month_ts))&set(list(np.array(weather['time'])))))
    weather = weather.sel(time=month_ts)
    weather['lat'] = np.round(weather['lat'].values, 3)
    weather['lon'] = np.round(weather['lon'].values, 3)
    for var in weather.data_vars:
        tempo[var] = weather[var]
    del weather
    print('weather done')

    radiation = xr.open_dataset(f'../../data/weather_data_samehr/radiation_data_{month_str}.nc')
    radiation = radiation.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"{var}_weather" for var in radiation.data_vars}
    radiation = radiation.rename(new_names)
    radiation = radiation.sel(time=month_ts)
    radiation['lat'] = np.round(radiation['lat'].values, 3)
    radiation['lon'] = np.round(radiation['lon'].values, 3)
    for var in radiation.data_vars:
        tempo[var] = radiation[var]
    del radiation

    geos_cf = xr.open_dataset(f'../../data/geos_cf_data/geos_cf_{month_str}.nc')
    geos_cf = geos_cf.drop_vars('TROPCOL_NO2')
    geos_cf = geos_cf.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"{var}_geoscf" for var in geos_cf.data_vars}
    geos_cf = geos_cf.rename(new_names)
    geos_cf = geos_cf.sel(time=month_ts)
    geos_cf['lat'] = np.round(geos_cf['lat'].values, 3)
    geos_cf['lon'] = np.round(geos_cf['lon'].values, 3)
    for var in geos_cf.data_vars:
        tempo[var] = geos_cf[var]
    del geos_cf

    NDVI = xr.open_dataset(f'../../data/NDVI_data/NDVI_{month_str}.nc')
    NDVI=NDVI[['NDVI']]
    NDVI['NDVI']=NDVI['NDVI'].fillna(-1)
    NDVI = NDVI.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    NDVI_times = np.array(NDVI['time'])
    NDVI_list = []
    for i in range(len(NDVI_times)):
        NDVI_temp=NDVI['NDVI'].isel(time=i)
        min_time = NDVI_times[i]
        if i+1<len(NDVI_times):
            max_time = NDVI_times[i+1]
            filtered_month_ts = month_ts[(month_ts >= min_time) & (month_ts <= max_time)]
        else:
            filtered_month_ts = month_ts[(month_ts >= min_time)]
        NDVI_temp = NDVI_temp.expand_dims({'time': filtered_month_ts})
        NDVI_temp = NDVI_temp.to_dataset(name='NDVI')
        NDVI_list.append(NDVI_temp)
    NDVI = xr.concat(NDVI_list, dim='time')
    NDVI['lat'] = np.round(NDVI['lat'].values, 3)
    NDVI['lon'] = np.round(NDVI['lon'].values, 3)
    for var in NDVI.data_vars:
        tempo[var] = NDVI[var]
    del NDVI

    emissions = xr.open_dataset(f'../../data/emissions/emissions_{month_str}.nc')
    emissions = emissions.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"{var}_emissions" for var in emissions.data_vars}
    emissions = emissions.rename(new_names)
    emissions = emissions.sel(time=month_ts)
    emissions['lat'] = np.round(emissions['lat'].values, 3)
    emissions['lon'] = np.round(emissions['lon'].values, 3)
    for var in emissions.data_vars:
        tempo[var] = emissions[var]
    del emissions


    elevation = xr.open_dataset(f'../../data/other/elevation.nc')
    elevation = elevation.drop_dims('time')
    elevation = elevation.expand_dims({'time': tempo['time']})
    elevation = elevation.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    elevation['lat'] = np.round(elevation['lat'].values, 3)
    elevation['lon'] = np.round(elevation['lon'].values, 3)
    for var in elevation.data_vars:
        tempo[var] = elevation[var]
    del elevation
    
    impervious_area = xr.open_dataset(f'../../data/other/impervious_area.nc')
    impervious_area = impervious_area.expand_dims({'time': tempo['time']})
    impervious_area = impervious_area.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    impervious_area['lat'] = np.round(impervious_area['lat'].values, 3)
    impervious_area['lon'] = np.round(impervious_area['lon'].values, 3)
    for var in impervious_area.data_vars:
        tempo[var] = impervious_area[var]
    del impervious_area
    
    population_density = xr.open_dataset(f'../../data/other/population_density.nc')
    population_density = population_density.expand_dims({'time': tempo['time']})
    population_density = population_density.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    population_density['lat'] = np.round(population_density['lat'].values, 3)
    population_density['lon'] = np.round(population_density['lon'].values, 3)
    for var in population_density.data_vars:
        tempo[var] = population_density[var]
    del population_density
    print('halfway')
    road_density = xr.open_dataset(f'../../data/other/road_density.nc')
    road_density = road_density.expand_dims({'time': tempo['time']})
    road_density = road_density.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    road_density['lat'] = np.round(road_density['lat'].values, 3)
    road_density['lon'] = np.round(road_density['lon'].values, 3)
    for var in road_density.data_vars:
        tempo[var] = road_density[var]
    del road_density
    
    water_map = xr.open_dataset(f'../../data/other/water_map.nc')
    water_map = water_map.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    water_map=water_map['LW'].isel(time=0)
    water_map = water_map.expand_dims({'time': tempo['time']})
    water_map = water_map.to_dataset(name='LW')
    new_names = {var: f"water_map_{var}" for var in water_map.data_vars}
    water_map = water_map.rename(new_names)
    water_map['lat'] = np.round(water_map['lat'].values, 3)
    water_map['lon'] = np.round(water_map['lon'].values, 3)
    for var in water_map.data_vars:
        tempo[var] = water_map[var]
    del water_map

    y_sensor = xr.open_dataset(f'../../data/y_sensor_only_o3/sensor_{month_str}.nc')
    y_sensor = y_sensor.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"y_sensor_{var}" for var in y_sensor.data_vars}
    y_sensor = y_sensor.rename(new_names)
    y_sensor = y_sensor.sel(time=month_ts)
    y_sensor['lat'] = np.round(y_sensor['lat'].values, 3)
    y_sensor['lon'] = np.round(y_sensor['lon'].values, 3)
    for var in y_sensor.data_vars:
        tempo[var] = y_sensor[var]
    del y_sensor

    y_sensor_mobile = xr.open_dataset(f'../../data/y_sensor_mobile_o3/sensor_mobile_{month_str}.nc')
    y_sensor_mobile = y_sensor_mobile.sel(lat=slice(lat_min, new_lat_max), lon=slice(lon_min, new_lon_max))
    new_names = {var: f"y_sensormobile_{var}" for var in y_sensor_mobile.data_vars}
    y_sensor_mobile = y_sensor_mobile.rename(new_names)
    y_sensor_mobile = y_sensor_mobile.sel(time=month_ts)
    y_sensor_mobile['lat'] = np.round(y_sensor_mobile['lat'].values, 3)
    y_sensor_mobile['lon'] = np.round(y_sensor_mobile['lon'].values, 3)
    for var in y_sensor_mobile.data_vars:
        tempo[var] = y_sensor_mobile[var]
    del y_sensor_mobile

    # print(tempo.isnull().any())
    for var in tempo.data_vars:
        try:
            # Access the variable by its name and modify its attributes
            tempo[var].attrs["bounds"] = tuple(tempo[var].attrs["bounds"])
            # print('done')
        except KeyError:
            # If 'bounds' attribute doesn't exist, do nothing
            pass
    print('merge_done')
    # tempo = tempo.load()  # load into memory, remove Dask backing
    tempo.to_netcdf(f'../../data/full_month_data_o3/model_{month_str}.nc', engine='netcdf4')
    del tempo

2025-05
weather done
halfway
merge_done
